In [31]:
#!/.conda/envs/learn python
# -*- coding: utf-8 -*-

"""
批量多类别精度评定

预测结果和真值标签均为栅格
文件名相同，尺寸相同，不需要地理坐标系
~~~~~~~~~~~~~~~~
code by wHy
Aerospace Information Research Institute, Chinese Academy of Sciences
wanghaoyu191@mails.ucas.ac.cn
"""

'\n批量多类别精度评定\n\n预测结果和真值标签均为栅格\n文件名相同，尺寸相同，不需要地理坐标系\n~~~~~~~~~~~~~~~~\ncode by wHy\nAerospace Information Research Institute, Chinese Academy of Sciences\nwanghaoyu191@mails.ucas.ac.cn\n'

In [32]:
import os
from statistics import mean
import sys
import fnmatch
import numpy as np
import gdal
import ogr
import osr
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [33]:
os.environ['GDAL_DATA'] = r'C:\Users\75198\.conda\envs\learn\Lib\site-packages\GDAL-2.4.1-py3.6-win-amd64.egg-info\gata-data' #防止报error4错误


model_names = ['DABNet', 'DE_Segformer', 'DeepLabv3_plus', 'FCN8S', 'HRNet', 'Segformer', 'UNetFormer', 'UNetPlusPlus']
cloud_removal_methods = ['0-ori','dark_channel_prior','clahe','ChaIR','SpA-GAN-dehaze']
# cloud_removal_methods = ['real_cloud','SpA_GAN-haze','ASM','ASM_NL']
ground_truth_pathes = ['1-raster_label_haze_lv1', '1-raster_label_haze_lv2_for_clear_Evaluation', '1-raster_label_haze_lv3']

output_txt_path = r'C:\Users\75198\OneDrive\0code\haze_code\cm_data_240426_Cloud_Removal.txt'
f = open(output_txt_path, 'w', encoding='UTF-8')

for lv in range(1,4):
    for model_name in model_names:
        for cloud_removal_method in cloud_removal_methods:

            ground_truth_path = r'E:\xinjiang_huyang_hongliu\Huyang_test_0808\1-raster_label/' + ground_truth_pathes[lv-1] # 存储真值标签的文件夹 真值标签应为栅格 带有地理坐标系 _for_clear_Evaluation
            predict_path = r'C:\Users\75198\OneDrive\论文\SCI-3-3 Remote sensing data augmentation\240408补充实验\全部除云算法\LV' + str(lv) + '/' + cloud_removal_method + '/' + 'predict_result_' + model_name # 存储预测栅格的文件 带有地理坐标系

            print(lv, cloud_removal_method, model_name)

            '''为了省事自动检索预测结果文件夹下的tif影像'''
            pre_pic_name = fnmatch.filter(os.listdir(predict_path), '*.tif')
            pre_pic_name_full = predict_path + '/' + pre_pic_name[0][:-4] + '.tif'

            im_data_pre_full = []
            im_data_true_full = []
            listpic = fnmatch.filter(os.listdir(ground_truth_path), '*.tif')
            for gt_name in listpic:
                gt_full_path = ground_truth_path + '/' + gt_name[:-4] + '.tif'
                pre_pic_name_full = predict_path + '/' + gt_name[:-4] + '.tif'
                '''以真值标签为参考，从预测栅格文件中裁剪出待评定部分'''
                # 获取真值标签地理坐标信息
                input_gt = gdal.Open(gt_full_path)

                # 获取预测结果数据
                input_pre = gdal.Open(pre_pic_name_full)

                # 定义裁剪窗口大小
                win_size = input_gt.RasterXSize

                # 整理数据
                im_data_pre = input_pre.ReadAsArray(0, 0, win_size, win_size)  # 读取预测结果对应区域的数据
                im_data_true = input_gt.ReadAsArray(0, 0, win_size, win_size) # 读取真值标签区域数据

                # print(im_data_pre.shape, im_data_true.shape)
                # print(type(im_data_pre))

                im_data_pre = list(im_data_pre.reshape(-1)) # 展平为一维
                im_data_true = list(im_data_true.reshape(-1)) # 展平为一维

                im_data_pre_full = im_data_pre_full + im_data_pre # 链接
                im_data_true_full = im_data_true_full + im_data_true

            # print(np.shape(im_data_pre_full), np.shape(im_data_true_full))
                
            unique_pre, count_pre=np.unique(im_data_pre_full,return_counts=True)
            data_count_pre=dict(zip(unique_pre, count_pre))
            unique_true,count_true=np.unique(im_data_true_full,return_counts=True)
            data_count_true=dict(zip(unique_true,count_true))
            # print(data_count_pre)
            # print(data_count_true)

            '''精度评定部分'''
            '''计算混淆矩阵'''
            cm = confusion_matrix(im_data_true_full, im_data_pre_full, normalize='true')
            print(cm)

            f.write(cloud_removal_method + ' ' + model_name + ' ' + 'lv'+ str(lv)+'\n')
            for i in range (len(cm)): # 打印混淆矩阵到
                f.write(str(cm[i])+'\n')
            f.write('\n')

f.close()




1 0-ori DABNet
[[0.9703842  0.01764666 0.01196914]
 [0.14114606 0.73291622 0.12593772]
 [0.09619154 0.06971292 0.83409554]]
1 dark_channel_prior DABNet
[[9.63256088e-01 3.62675566e-02 4.76355137e-04]
 [1.07262182e-01 8.89491269e-01 3.24654857e-03]
 [1.69709081e-01 8.00468852e-01 2.98220668e-02]]
1 clahe DABNet
[[9.78832327e-01 2.03354860e-02 8.32186685e-04]
 [1.88898695e-01 8.09872029e-01 1.22927567e-03]
 [2.49529226e-01 6.51281657e-01 9.91891165e-02]]
1 ChaIR DABNet
[[0.96979019 0.02738612 0.0028237 ]
 [0.12421988 0.86054025 0.01523987]
 [0.16263787 0.45278813 0.384574  ]]
1 SpA-GAN-dehaze DABNet
[[0.96623761 0.02946084 0.00430154]
 [0.11848326 0.86804198 0.01347475]
 [0.14472926 0.49175666 0.36351408]]
1 0-ori DE_Segformer
[[0.98151398 0.01848602 0.        ]
 [0.10089516 0.89910484 0.        ]
 [0.36674225 0.63325775 0.        ]]
1 dark_channel_prior DE_Segformer
[[0.73059534 0.26940466 0.        ]
 [0.03166173 0.96833827 0.        ]
 [0.04630875 0.95369125 0.        ]]
1 clahe DE_Se